# Benchmarking Automation Speech Recognition Models

## Kaggle Specific

In [1]:
!pip install --quiet --upgrade transformers datasets accelerate timm datasets[audio]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.1/367.1 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 69.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install --quiet evaluate jiwer mistral-common bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 104.3 MB/s eta 0:00:00


In [3]:
# Setup ugging Face
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")

login(token=HF_TOKEN)

## Dependencies

In [ ]:
import json
import os
import re
import time
from pathlib import Path

import numpy as np
import pandas as pd
import torch
from datasets import load_dataset
from evaluate import load
from torchinfo import summary
from tqdm.auto import tqdm
from transformers import WhisperForConditionalGeneration, WhisperProcessor, BitsAndBytesConfig

2025-07-30 15:54:54.426783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753890894.670976      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753890894.742180      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Utilities

In [5]:
main_path = Path("/kaggle/working")

In [6]:
def normalize_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)  # remove punctuation
    text = re.sub(r"\s+", " ", text)  # normalize whitespace
    text = text.strip()

    return text


@torch.inference_mode()
def predict_whisper(model, processor, sample):
    result = {
        "id": sample["id"],
        "reference": normalize_text(sample["text"]),
    }

    audio = sample["audio"]
    inputs = processor(
        audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt"
    )
    inputs = inputs.to(model.device, dtype=torch.float16)

    outputs = model.generate(**inputs, do_sample=False)
    predicted_ids = outputs[0]
    transcription = processor.decode(predicted_ids)

    result["prediction"] = normalize_text(transcription)

    return result

In [7]:
def benchmark(model, processor, predict_fn, dataset, max_samples: int | None = None):
    benchmark_results = []
    wer = load("wer")

    for i, sample in enumerate(tqdm(dataset, desc="Benchmarking", total=max_samples)):
        if max_samples is not None and i >= max_samples:
            break

        start_time = time.perf_counter()
        prediction = predict_fn(model, processor, sample)

        end_time = time.perf_counter()

        inference_time = end_time - start_time
        wer_result = 100 * wer.compute(
            references=[prediction["reference"]],
            predictions=[prediction["prediction"]],
        )

        benchmark_results.append(
            {**prediction, "inference_time": inference_time, "wer": wer_result}
        )

    total_samples = len(benchmark_results)
    average_inference_time = np.mean(
        [result["inference_time"] for result in benchmark_results]
    ).item()
    average_wer = np.mean([result["wer"] for result in benchmark_results]).item()

    benchmark_summary = {
        "total_samples": total_samples,
        "average_inference_time": average_inference_time,
        "average_wer": average_wer,
    }

    result = {
        "benchmark_results": benchmark_results,
        "benchmark_summary": benchmark_summary,
    }

    return result

In [8]:
def save_results(result, folder_path: Path):
    benchmark_results = pd.DataFrame(result["benchmark_results"])
    benchmark_summary = result["benchmark_summary"]

    os.makedirs(folder_path, exist_ok=True)

    benchmark_results.to_csv(folder_path / "benchmark_results.csv", index=False)

    with open(folder_path / "benchmark_summary.json", "w") as f:
        json.dump(benchmark_summary, f, indent=4)

## Dataset

In [9]:
dataset = load_dataset("openslr/librispeech_asr", "clean", split="test", streaming=True)

README.md: 0.00B [00:00, ?B/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/48 [00:00<?, ?it/s]

## Models

### openai/whisper-large-v2

In [10]:
whisper_processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")

# NormalFloat 4-bit is highly optimized for weights in neural networks with a normal distribution
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
whisper_model_4_bit = WhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-large-v2",
    quantization_config=quantization_config,
    device_map="auto"
)

preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [11]:
summary(whisper_model_4_bit)

Layer (type:depth-idx)                             Param #
WhisperForConditionalGeneration                    --
├─WhisperModel: 1-1                                --
│    └─WhisperEncoder: 2-1                         --
│    │    └─Conv1d: 3-1                            308,480
│    │    └─Conv1d: 3-2                            4,916,480
│    │    └─Embedding: 3-3                         (1,920,000)
│    │    └─ModuleList: 3-4                        315,064,320
│    │    └─LayerNorm: 3-5                         2,560
│    └─WhisperDecoder: 2-2                         --
│    │    └─Embedding: 3-6                         66,387,200
│    │    └─WhisperPositionalEmbedding: 3-7        573,440
│    │    └─ModuleList: 3-8                        420,126,720
│    │    └─LayerNorm: 3-9                         2,560
├─Linear: 1-2                                      66,387,200
Total params: 875,688,960
Trainable params: 138,987,520
Non-trainable params: 736,701,440

In [12]:
!nvidia-smi

Wed Jul 30 15:56:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   37C    P0             32W /  250W |    2157MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
results = benchmark(
    whisper_model_4_bit, whisper_processor, predict_whisper, dataset, max_samples=1000
)
output_folder = main_path / "openai/whisper-large-v2/4-bit" 
save_results(results, output_folder)

Benchmarking:   0%|          | 0/1000 [00:00<?, ?it/s]

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
